In [197]:
%load_ext cython

import numpy as np
import cython

from isocedar.datasets import MalanchevDataset
from sklearn.ensemble import IsolationForest

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


* Dataset is an o-by-f array, where o is objects and f is features.
* Labels is a one-dimentional o array with three types of labels:
 * -1 for anomalies,
 * 0 for unknowns,
 * 1 for regular data.

In [198]:
class RandomPineForest:
    def __init__(self, trees=100, subsamples=256, depth=None, seed=0):
        self.subsamples = subsamples
        self.trees = trees
        self.depth = depth
        
        self.seedseq = np.random.SeedSequence(seed)
        self.rng = np.random.default_rng(seed)
        
        self.estimators = []
        self.n = 0

    def fit(self, data):
        n = data.shape[0]
        self.n = n
        self.subsamples = self.subsamples if n > self.subsamples else n
        
        self.depth = self.depth or int(np.ceil(np.log2(self.subsamples)))

        self.estimators = [None] * self.trees
        seeds = self.seedseq.spawn(self.trees)
        for i in range(self.trees):
            subs = self.rng.choice(n, self.subsamples)
            gen = RandomPineGenerator(data[subs, :], self.depth, seeds[i])
            self.estimators[i] = gen.pine
        
        return self

    def mean_paths(self, data):
        means = np.zeros(data.shape[0])
        for ti in range(self.trees):
            path = self.estimators[ti].paths(data)
            means += path
        
        means /= self.trees
        return means

    def scores(self, data):
        means = self.mean_paths(data)
        return - 2 ** (-means / average_path_length(self.subsamples))


class RandomPine:
    def __init__(self, features, selectors, values):
        self.features = features
        self.len = selectors.shape[0]
        
        # Two complementary arrays.
        # Selectors select feature to branch on.
        self.selectors = selectors
        # Values either set the deciding feature value or set the closing path length
        self.values = values

    def _get_one_path(self, key):
        i = 1
        while 2 * i < self.selectors.shape[0]:
            f = self.selectors[i]
            if f < 0:
                break
            
            if key[f] <= self.values[i]:
                i = 2 * i
            else:
                i = 2 * i + 1

        return self.values[i]
    
    def paths(self, x):
        n = x.shape[0]
        paths = np.empty(n)
        for i in range(n):
            paths[i] = self._get_one_path(x[i, :])
        
        return paths


class RandomPineGenerator:
    def __init__(self, sample, depth, seed=0):
        self.depth = depth
        self.features = sample.shape[1]
        self.length = 1 << (depth + 1)
        self.rng = np.random.default_rng(seed)
        self.selectors = np.full(self.length, -1, dtype=np.int32)
        self.values = np.full(self.length, 0, dtype=np.float64)
        
        self._populate(1, sample)
        
        self.pine = RandomPine(self.features, self.selectors, self.values)
    
    def _populate(self, i, sample):
        
        if sample.shape[0] == 1:
            self.values[i] = np.floor(np.log2(i))
            return
        
        if self.length <= 2 * i:
            self.values[i] = np.floor(np.log2(i)) + \
                average_path_length(sample.shape[0])
            return
        
        selector = self.rng.integers(self.features)
        self.selectors[i] = selector

        minval = np.min(sample[:, selector])
        maxval = np.max(sample[:, selector])
        if minval == maxval:
            self.selectors[i] = -1
            self.values[i] = np.floor(np.log2(i)) + \
                average_path_length(sample.shape[0])
            return

        value = self.rng.uniform(minval, maxval)
        self.values[i] = value

        self._populate(2 * i, sample[sample[:, selector] <= value])
        self._populate(2 * i + 1, sample[sample[:, selector] > value])


def average_path_length(n):
    if n <= 1:
        return 0
    elif n == 2:
        return 1
    else:
        return 2.0 * (np.log(n - 1.0) + np.euler_gamma) - 2.0 * (n - 1.0) / n

In [235]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp -a

import numpy as np
from cython.parallel cimport prange, parallel

cimport numpy as np
cimport cython


cdef packed struct selector_t:
    np.int32_t feature
    np.int32_t left
    np.float64_t value
    np.int32_t right


def calc_mean_paths(selectors, indices, data):
    paths = np.zeros(data.shape[0])
    # TODO: check data consistency
    _mean_paths(selectors, indices, data, paths)
    return paths


@cython.boundscheck(False)
@cython.wraparound(False)
cdef void _mean_paths(selector_t [::1] selectors,
                         np.int64_t [::1] indices,
                         np.double_t [:, ::1] data,
                         np.double_t [::1] paths):

    cdef Py_ssize_t trees
    cdef Py_ssize_t tree_index
    cdef Py_ssize_t x_index
    cdef selector_t selector
    cdef Py_ssize_t tree_offset
    cdef np.int32_t feature, i
    
    with nogil, parallel():
        trees = indices.shape[0] - 1
        
        for x_index in prange(data.shape[0], schedule='static'):
            for tree_index in range(trees):
                tree_offset = indices[tree_index]
                i = 0
                while True:
                    selector = selectors[tree_offset + i]
                    feature = selector.feature
                    if feature < 0:
                        break

                    if data[x_index, feature] <= selector.value:
                        i = selector.left
                    else:
                        i = selector.right

                paths[x_index] += selector.value

            paths[x_index] /= trees

In [298]:
import numpy as np


class ForestEvaluator:
    selector_dtype = np.dtype([('feature', np.int32), ('left', np.int32), ('value', np.double), ('right', np.int32)])
    
    def __init__(self, samples, selectors, indices):
        self.samples = samples

        self.selectors = selectors
        self.indices = indices

    @classmethod
    def combine_selectors(cls, selectors_list):
        lens = [len(sels) for sels in selectors_list]
        full_len = sum(lens)
        
        selectors = np.empty((full_len,), dtype=cls.selector_dtype)
        
        indices = np.empty((len(selectors_list) + 1,), dtype=np.int64)
        indices[0] = 0
        indices[1:] = np.add.accumulate(lens)

        for i in range(len(selectors_list)):
            selectors[indices[i]:indices[i + 1]] = selectors_list[i]
        
        return selectors, indices

    def score_samples(self, x):
        return -2 ** (- calc_mean_paths(self.selectors, self.indices, x) / average_path_length(self.samples))


class PineEvaluator(ForestEvaluator):
    def __init__(self, pine_forest):
        pines = pine_forest.estimators
        self.trees = len(pines)
        if self.trees < 1:
            raise ValueError('a forest without trees?')

        selectors, indices = self.combine_selectors(
            [self.extract_selectors(pine) for pine in pines])

        super(PineEvaluator, self).__init__(
            samples = pine_forest.subsamples,
            selectors = selectors,
            indices = indices)
    
    @classmethod
    def extract_selectors(cls, pine):
        selectors = np.zeros((pine.len,), dtype=cls.selector_dtype)

        mapping = np.full((pine.len,), -1, dtype=np.int32)
        current = 0
        for i in range(1, pine.len):
            if pine.selectors[i] != -1 or pine.values[i] != 0:
                mapping[i] = current
                current += 1
        
        selectors = selectors[:current]
        
        for i in range(1, pine.len):
            if pine.selectors[i] != -1 or pine.values[i] != 0:
                current = mapping[i]
                
                feature = pine.selectors[i]
                selectors[current]['feature'] = feature
                selectors[current]['value'] = pine.values[i]
                
                if 2 * i >= pine.len:
                    continue
                
                selectors[current]['left'] = mapping[2 * i]
                selectors[current]['right'] = mapping[2 * i + 1]
        
        return selectors


class IsoforestEvaluator(ForestEvaluator):
    
    def __init__(self, isoforest):
        selectors_list = [self.extract_selectors(e) for e in isoforest.estimators_]
        selectors, indices = self.combine_selectors(selectors_list)

        super(IsoforestEvaluator, self).__init__(
            samples=isoforest.max_samples_,
            selectors=selectors,
            indices=indices)

    @classmethod
    def extract_selectors(cls, estimator):
        nodes = estimator.tree_.__getstate__()['nodes']
        selectors = np.zeros_like(nodes, dtype=cls.selector_dtype)

        selectors['feature'] = nodes['feature']
        selectors['feature'][selectors['feature'] < 0] = -1

        selectors['left'] = nodes['left_child']
        selectors['right'] = nodes['right_child']
        selectors['value'] = nodes['threshold']
        
        n_node_samples = nodes['n_node_samples']
        
        def correct_values(i, depth):
            if selectors[i]['feature'] < 0:
                selectors[i]['value'] = depth + average_path_length(n_node_samples[i])
            else:
                correct_values(selectors[i]['left'], depth + 1)
                correct_values(selectors[i]['right'], depth + 1)
    
        correct_values(0, 0)
    
        return selectors

In [304]:
dataset = MalanchevDataset(inliers=2**20, outliers=2**6)

In [277]:
%%time
forest = RandomPineForest().fit(dataset.data)

CPU times: user 343 ms, sys: 2.89 ms, total: 346 ms
Wall time: 349 ms


In [278]:
%%time
pe = PineEvaluator(forest)

CPU times: user 253 ms, sys: 1.84 ms, total: 255 ms
Wall time: 255 ms


In [282]:
%%time
pescores = pe.score_samples(dataset.data)

CPU times: user 282 ms, sys: 0 ns, total: 282 ms
Wall time: 101 ms


In [280]:
%%time
scores = forest.scores(dataset.data)

CPU times: user 29.7 s, sys: 0 ns, total: 29.7 s
Wall time: 29.7 s


In [305]:
%%time
isoforest = IsolationForest().fit(dataset.data)

CPU times: user 1.45 s, sys: 8.61 ms, total: 1.46 s
Wall time: 1.46 s


In [306]:
%%time
isoscores = isoforest.score_samples(dataset.data)

CPU times: user 23.5 s, sys: 3.52 s, total: 27 s
Wall time: 27 s


In [307]:
%%time
peiso = IsoforestEvaluator(isoforest)

CPU times: user 134 ms, sys: 86 µs, total: 134 ms
Wall time: 131 ms


In [308]:
%%time
peisoscores = peiso.score_samples(dataset.data)

CPU times: user 10.4 s, sys: 4.78 ms, total: 10.4 s
Wall time: 2.67 s
